In [4]:
import pandas as pd
from pathlib import Path
from core import ROOT_PATH

In [5]:
dataset_path = ROOT_PATH / Path("data/df_clusterized.pkl")
df = pd.read_pickle(dataset_path)
df.head()

,cluster_label,emb,target
0,5.0,"[-0.19262028, 0.67613167, -0.06970257, 1.36109...",90000.0
1,5.0,"[-0.18461296, 0.79767823, -0.011102157, 1.4939...",90000.0
2,5.0,"[-0.20171373, 0.72063917, -0.051841334, 1.3009...",70000.0
3,12.0,"[-0.23105316, 1.0083596, 0.08954958, 0.7279827...",32500.0
4,12.0,"[-0.23105316, 1.0083596, 0.08954958, 0.7279827...",32500.0


In [6]:
dict_of_dfs = {key: val for key, val in df.groupby("cluster_label")}
dict_of_dfs.keys()

dict_keys([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0])

In [7]:
%load_ext autoreload
%autoreload 2

from core.models.clustering_model import StackedModels, hyperparameters_tuning

In [8]:
# params = {
#     "iterations": 10,
#     "learning_rate": 0.07060926866932606,
#     "depth": 10,
#     "subsample": 0.710251337503286,
#     "colsample_bylevel": 0.6883135875379539,
#     "min_data_in_leaf": 1,
# }
# 
# sample_model = StackedModels(**params)
# sample_model = sample_model.train(dict_of_dfs, test_size=0.2)

# # or

# params = hyperparameters_tuning(dict_of_dfs)


In [9]:
params = {
    "iterations": 10,
    "learning_rate": 0.07060926866932606,
    "depth": 10,
    "subsample": 0.710251337503286,
    "colsample_bylevel": 0.6883135875379539,
    "min_data_in_leaf": 1,
}

sample_model = StackedModels(**params)

split_dataset_dict = sample_model.split_dict_dataset(dict_of_dfs, test_size=0.2)
train_dataset_dict = {k: v for k, (v, _) in split_dataset_dict.items()}
test_dataset_dict = {k: v for k, (_, v) in split_dataset_dict.items()}

sample_model = sample_model.train(train_dataset_dict, test_size=0.0)

metric = sample_model.evaluate(dataset_dict=test_dataset_dict)
print(metric)

{}
0:	learn: 47577.6432816	total: 75.3ms	remaining: 678ms
1:	learn: 46589.0345788	total: 88ms	remaining: 352ms
2:	learn: 45559.7078350	total: 119ms	remaining: 279ms
3:	learn: 44517.1554869	total: 142ms	remaining: 212ms
4:	learn: 43550.4166412	total: 168ms	remaining: 168ms
5:	learn: 42595.0620505	total: 184ms	remaining: 123ms
6:	learn: 41820.4292491	total: 191ms	remaining: 81.7ms
7:	learn: 41008.1860314	total: 198ms	remaining: 49.5ms
8:	learn: 40267.0168294	total: 204ms	remaining: 22.7ms
9:	learn: 39596.0872044	total: 210ms	remaining: 0us
Model trained
{}
0:	learn: 10988.8826154	total: 638us	remaining: 5.74ms
1:	learn: 10612.6345103	total: 1.07ms	remaining: 4.3ms
2:	learn: 10301.9604146	total: 1.73ms	remaining: 4.04ms
3:	learn: 10034.5434144	total: 2.25ms	remaining: 3.38ms
4:	learn: 9787.2140010	total: 2.81ms	remaining: 2.81ms
5:	learn: 9510.3827502	total: 3.17ms	remaining: 2.11ms
6:	learn: 9244.1034293	total: 3.62ms	remaining: 1.55ms
7:	learn: 9000.7209697	total: 4.67ms	remaining: 1.17

In [10]:
print(sample_model.evaluate(test_dataset_dict))
sample_model.save_model(ROOT_PATH / "checkpoints/stacked_model.pkl")


{
    "0.0": 26031.076855614414,
    "1.0": 3391.46210675567,
    "2.0": 21199.02532730562,
    "3.0": 11933.120317095689,
    "4.0": 32961.47497209855,
    "5.0": 32200.171914251558,
    "6.0": 37756.44882186081,
    "7.0": 9479.600131278366,
    "8.0": 15129.827942679487,
    "9.0": 13278.413763894805,
    "10.0": 23777.910978376927,
    "11.0": 19151.38677169454,
    "12.0": 24726.813258664995,
    "13.0": 21438.627847467087,
    "14.0": 14612.137875745993,
    "15.0": 15733.502938466101,
    "16.0": 18006.504630665087
}
20047.500379642097
Model saved at /Users/Konstantin.Grotov/Documents/programming/projects/jbr/salary-prediction/checkpoints/stacked_model.pkl


In [11]:
sample_model = sample_model.load_model(ROOT_PATH / "checkpoints/stacked_model.pkl")
sample_model.evaluate(test_dataset_dict)

Model successfully loaded
{
    "0.0": 26031.076855614414,
    "1.0": 3391.46210675567,
    "2.0": 21199.02532730562,
    "3.0": 11933.120317095689,
    "4.0": 32961.47497209855,
    "5.0": 32200.171914251558,
    "6.0": 37756.44882186081,
    "7.0": 9479.600131278366,
    "8.0": 15129.827942679487,
    "9.0": 13278.413763894805,
    "10.0": 23777.910978376927,
    "11.0": 19151.38677169454,
    "12.0": 24726.813258664995,
    "13.0": 21438.627847467087,
    "14.0": 14612.137875745993,
    "15.0": 15733.502938466101,
    "16.0": 18006.504630665087
}


20047.500379642097